In [1]:
import pandas as pd
from pyproj import Proj, transform
import fiona
from fiona.crs import from_epsg
import logging
import sys


# outProj = Proj(init='epsg:3857')
# inProj = Proj(init='epsg:4326') # It is equal to WSG

In [3]:
shape = fiona.open("mp/Uso_das_Terras_e_Cobertura_Vegetal__1984.shp")
print (shape.schema)
# Chico Mendes é o id 73

{'properties': OrderedDict([('OBJECTID', 'int:4'), ('A84', 'float:24.15'), ('CLASSE', 'str:28'), ('RuleID', 'str:28')]), 'geometry': 'Polygon'}


In [5]:
shape[1713]

{'type': 'Feature',
 'id': '1713',
 'properties': OrderedDict([('OBJECTID', 1714),
              ('A84', 7423157.52783203),
              ('CLASSE', 'Área Urbana'),
              ('RuleID', 'Área Urbana')]),
 'geometry': {'type': 'Polygon',
  'coordinates': [[(661540.4111000001, 7455293.0789),
    (661634.4737, 7455269.5789),
    (661667.4737999998, 7455224.5788),
    (661654.2237, 7455071.0786),
    (661393.8483999996, 7455040.0786),
    (661466.6608999996, 7454545.5779),
    (661526.5983999996, 7454524.5779),
    (661518.8483999996, 7454452.5778),
    (661502.7233999996, 7454471.5778),
    (661463.9732999997, 7454459.0778),
    (661425.2232999997, 7454427.5778),
    (661412.2857999997, 7454390.0777),
    (661380.0356999999, 7454346.0777),
    (661341.2856999999, 7454346.0777),
    (661289.5981000001, 7454358.5777),
    (661244.4105000002, 7454383.5778),
    (661154.0354000004, 7454459.0779),
    (661076.5352999996, 7454452.5779),
    (660986.1601999998, 7454377.5778),
    (660882.847

In [6]:
# Transformar escala lag, long em Metros no globo achatado

# salvando o esquema e pegando a projecao de entrada
sink_schema = shape.schema.copy()
p_in = Proj(shape.crs)

# definindo um novo shape de saida, no mesmo local do shape atual com a projeção mudada
with fiona.open(
        'area_1984.shp', 'w',
        crs=from_epsg(3857),
        driver=shape.driver,
        schema=sink_schema,
        ) as sink:

    p_out = Proj(sink.crs)

    # fazendo apenas para o Chico Mendes (esse arquivo tem Polygon e MultiPolygon)
    # nesse caso, se fizesse para o arquivo todo, nao seria contemplado alguns shapes
    f = shape[1713]

    try:
        # verifica se eh da estrutura poligono
        assert f['geometry']['type'] == "Polygon"
        new_coords = []
        # para todas as coordenadas do poligono
        for ring in f['geometry']['coordinates']:
            x2, y2 = transform(p_in, p_out, *zip(*ring)) #transforma
            new_coords.append(zip(x2, y2)) # salva a nova lista de coordenadas
        f['geometry']['coordinates'] = new_coords
        sink.write(f)

    except Exception:
        # Writing uncleanable features to a different shapefile
        # is another option.
        logging.exception("Error transforming feature %s:", f['id'])

/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/opt/anaconda/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [6]:
print(chico['geometry']['coordinates'])

[[(-43.46900242299945, -23.02168791899976), (-43.46900242299945, -23.02168791899976), (-43.46868186399948, -23.021686624999813), (-43.46840067299947, -23.021688170999763), (-43.468267365999466, -23.02169125999977), (-43.46813439599948, -23.021700530999794), (-43.46800209399948, -23.021715959999764), (-43.46787079399946, -23.021737508999752), (-43.46774082699944, -23.021765122999785), (-43.467612517999456, -23.021798733999784), (-43.46748619099947, -23.0218382559998), (-43.46736216399946, -23.02188358899978), (-43.46724074699945, -23.021934621999776), (-43.46712224799944, -23.02199122399979), (-43.467006961999466, -23.02205325399979), (-43.46689518199948, -23.022120554999788), (-43.466815307999475, -23.022174851999804), (-43.46673930899943, -23.02223372299978), (-43.46666748999946, -23.022296928999744), (-43.466600137999436, -23.022364216999783), (-43.46653752399943, -23.022435317999786), (-43.46647989999948, -23.022509945999776), (-43.46642749699948, -23.022587800999776), (-43.46638052

In [7]:
shape2 = fiona.open("area_1984.shp")
print (shape2.schema)

{'properties': OrderedDict([('OBJECTID', 'int:4'), ('A84', 'float:24.15'), ('CLASSE', 'str:28'), ('RuleID', 'str:28')]), 'geometry': 'Polygon'}


In [9]:
print(shape2)

<open Collection 'area_1984.shp:area_1984', mode 'r' at 0x7f5066662a10>
